In [23]:
%pip install ja_stopword_remover

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import MeCab
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from nltk import ngrams

nlp = spacy.load('ja_ginza')
ruler = EntityRuler(nlp)

# 特許データフレーム

In [2]:
df = pd.read_excel("dwwmcc429001-20210628-143258-30187.xlsx",index_col=0,header=2)

In [3]:
df.columns

Index(['出願番号', '筆頭ＩＰＣ(最新)', '筆頭ＦＩ(最新)', 'ＦＩ(最新)', 'テーマコード(最新)', 'Fターム(最新)',
       '発明の名称', '公報番号', '被引用文献番号', '出願人(最新)', '発明者(最新)', '出願日(遡)', '公開(表)日',
       '被引用数', '要約', '発明が解決しようとする課題', '発明の効果', '産業上の利用分野', '従来技術'],
      dtype='object')

In [4]:
df

,出願番号,筆頭ＩＰＣ(最新),筆頭ＦＩ(最新),ＦＩ(最新),テーマコード(最新),Fターム(最新),発明の名称,公報番号,被引用文献番号,出願人(最新),発明者(最新),出願日(遡),公開(表)日,被引用数,要約,発明が解決しようとする課題,発明の効果,産業上の利用分野,従来技術
No.,,,,,,,,,,,,,,,,,,,
1,特願2021-32103,E04B1/348,E04B1/348@H,E04B1/348@H,2000000000000000074681867494291977943878655150...,－,箱型建築物ユニットおよび建築物,特開2021-88925,－,惠美須 健也,惠美須 健也,2018/8/1,2021/6/10,－,【課題】ＣＬＴパネル同士を容易に接合して箱型建築物ユニットの提供。【解決手段】直方体の各面の...,【発明が解決しようとする課題】【０００６】特許文献１に記載の建築物は、特段の連結具を使用する...,【発明の効果】【００１２】（１）直方体の各面の直交部は、第１のＣＬＴパネルの内側主面に対して...,【技術分野】【０００１】本発明は、直交集成板（ＣＬＴ）からなるパネル（ＣＬＴパネル）により形...,【背景技術】【０００２】直交集成板（ＣＬＴ（Ｃｒｏｓｓ Ｌａｍｉｎａｔｅｄ Ｔｉｍｂｅｒ...
2,特願2021-29620,E05B49/00,E05B49/00@J,E05B49/00@J,1999999999999999842193666166429405315108085538...,2E250AA02|2E250BB08|2E250BB35|2E250CC20|2E250D...,錠制御システム、錠制御方法、ユーザ端末及び錠制御プログラム,特開2021-88924,－,ＫＤＤＩ株式会社,岸 利光,2019/3/19,2021/6/10,－,【課題】錠開閉機器の消費電力量を低減させる。【解決手段】錠制御システムＳは、錠を開閉する錠開...,【発明が解決しようとする課題】【０００４】ところで、錠開閉機器をはじめとするＩｏＴ機器におい...,【発明の効果】【００１４】本発明によれば、錠開閉機器の消費電力量を低減させることができるとい...,【技術分野】【０００１】本発明は、錠開閉機器を制御する錠制御システム、錠制御方法、ユーザ端末...,【背景技術】【０００２】近年、ＩｏＴ（Internet of Things）の普及により、ネ...
3,特願2021-27142,E04C2/26,E04C2/26@R,E04C2/26@R|E04B1/94@V|E04B9/00@D|E04B9/04@B,2E162|2E104|2E001,2E001DE01|2E001FA06|2E001FA10|2E001FA14|2E001G...,内装パネル,特開2021-80828,－,パナソニックＩＰマネジメント株式会社,梅岡 一哲|前田 直彦|田中 恒也|岡田 芳弘|新田 悠吉|多田 昇平,2018/8/30,2021/5/27,－,【課題】軽量化を図りながらも、剛性を向上し得る内装パネルを提供する。【解決手段】内装パネル１...,【発明が解決しようとする課題】【０００４】しかしながら、上記のような断熱パネルからなる天井パ...,【発明の効果】【０００７】本発明に係る内装パネルは、上述のような構成としたことで、軽量化を図...,【技術分野】【０００１】本発明は、建物の天井や壁などを構成する内装パネルに関する。,【背景技術】【０００２】従来より、吊天井に用いられる天井パネルなどの内装パネルは、ロックウー...
4,特願2021-4804,C22C9/04,C22C9/04,C22C9/04|B22F3/105|B22F3/16|B33Y10/00|B33Y70/0...,4K018|2D060|4F213,2D060BE01|2D060BF01|4K018AA03|4K018BA02|4K018C...,銅合金、銅合金の使用、及び水栓金具を生産する方法,特開2021-73374,－,株式会社ＬＩＸＩＬ,カルステン・ロマノウスキー,2018/8/13,2021/5/13,－,【課題】本発明は、銅合金、付加製造プロセスに対応する銅合金の使用、対応する銅合金で作製された...,－,－,－,－
5,特願2021-1095,E04B2/70,E04B2/70,E04B2/70,200,2E002FA03|2E002LB05|2E002LC04|2E002LC06|2E002MA07,ログハウス用木材,特開2021-50601,－,株式会社フェニックスホーム|松下 勝久,松下 勝久,2018/7/31,2021/4/1,－,【課題】セトリングを抑制できるログハウス用木材を供すること。【解決手段】それぞれ一方向に並行...,【発明が解決しようとする課題】【０００３】しかし、従来のラミネート材はセトリングと呼ばれる収...,【発明の効果】【０００９】本発明によれば、内側木材の繊維方向にログハウス用木材が積み上げられ...,【技術分野】【０００１】本発明は、ログハウス用木材に関する。,【背景技術】【０００２】近年、ログハウス用木材として、丸太を中心から切り分けて、それらを背中...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,特願2020-501129,E06C1/39,E06C1/39@A,E06C1/39@A|E06C7/14,200000000000000017642722810612845281403731968,2E044AA04|2E044BA06|2E044BA11|2E044BB01|2E044B...,製造設備の構成要素ラックシステムを有する踏み台,特表2020-527659,－,ラム リサーチ コーポレーション|ＬＡＭ ＲＥＳＥＡＲＣＨ ＣＯＲＰＯＲＡＴＩＯＮ,バスケス・ミゲル・ベンジャミン|スレビン・ダミエン,2018/7/6,2020/9/10,－,踏み台アセンブリが提供される。踏み台アセンブリは、製造設備においてモジュールの側面に接続する...,－,－,【技術分野】【０００１】本開示の実施態様は、製造設備において使用される踏み台、並びに、関連す...,【背景技術】【０００２】半導体製造設備におけるスペースはコストがかかるため、製造業者は、ツー...
497,特願2020-500837,C09K8/68,C09K8/68,C09K8/68|E21B43/22@A,4H022|2D129,－,表面改質を使用するコンデンセートバンキングの緩和,特表2020-527626,－,サウジ アラビアン オイル カンパニー,サイード， モハメッド|リャン， フェン|オウ， ホイスウェン|コックス， ジェイソン,2018/7/18,2020/9/10,－,本出願は、コンデンセートバンキングを緩和するための方法およびシステムに関する。一部の実施形態...,－,－,【技術分野】【０００１】関連出願への相互参照本出願は、２０１７年７月２０日に出願された米国仮...,【背景技術】【０００３】背景ガスコンデンセート貯留層は、エネルギー資源に対する世界的な需要の...
498,特願2020-500800,B63B21/29,B63B21/29,B63B21/29|E02B3/20@A,3D116|2D022,－,浮遊構造体を繋ぎ留めるための係留装置,特表2020-526444,－,ソレタンシュ フレシネ|ＳＯＬＥＴＡＮＣＨＥ ＦＲＥＹＳＳＩＮＥＴ,ジャン・ダニエル・ルボン|フランソワ・テプリー,2018/7/10,2020/8/31,－,本発明は、沈められるように、および、浮遊構造体を繋ぎ留めるように意図されている係留装置のため...,【発明が解決しようとする課題】【０００８】本発明はこの状況を改善することを目的とする。,－,【技術分野】【０００１】本発明の分野は、浮遊構造体の繋ぎ留めに関し、具体的には、この目的のた...,【背景技術】【０００２】いくつかの方策が、例えば石油事業構造体または風力タービン式の電力生産...


In [5]:
df['従来技術']

No.
1      【背景技術】【０００２】直交集成板（ＣＬＴ（Ｃｒｏｓｓ  Ｌａｍｉｎａｔｅｄ  Ｔｉｍｂｅｒ...
2      【背景技術】【０００２】近年、ＩｏＴ（Internet of Things）の普及により、ネ...
3      【背景技術】【０００２】従来より、吊天井に用いられる天井パネルなどの内装パネルは、ロックウー...
4                                                      －
5      【背景技術】【０００２】近年、ログハウス用木材として、丸太を中心から切り分けて、それらを背中...
                             ...                        
496    【背景技術】【０００２】半導体製造設備におけるスペースはコストがかかるため、製造業者は、ツー...
497    【背景技術】【０００３】背景ガスコンデンセート貯留層は、エネルギー資源に対する世界的な需要の...
498    【背景技術】【０００２】いくつかの方策が、例えば石油事業構造体または風力タービン式の電力生産...
499    【背景技術】【０００２】無機フィラーを含む樹脂層を芯材シートとし、該芯材シートの両面に金属シ...
500    【背景技術】【０００２】現在使用されているフリーラジカル硬化性留め付けコーク（コーキング材）...
Name: 従来技術, Length: 500, dtype: object

# 目的センテンス

In [6]:
sent = df['従来技術'][1]
print(sent)

【背景技術】【０００２】直交集成板（ＣＬＴ（Ｃｒｏｓｓ  Ｌａｍｉｎａｔｅｄ  Ｔｉｍｂｅｒ））は、板の繊維方向が各層で互いに直交するように積層接着したパネルである。ＣＬＴパネルは、直交積層であるため、建築の資材として使用されてきた集成材よりも強度が安定しており、変形しにくく、コンクリートにも匹敵する強度を持ち、断熱性にも優れているという特徴がある。【０００３】従来、このＣＬＴパネルを利用した住宅構造が提案されている。例えば、特許文献１には、２以上のＣＬＴパネル等の箱型パネルを自重により支持されるように配置し、特段の連結具を使用することなく建築物を建築することが記載されている。なお、特許文献１には、丁番等の連結具を利用したり、接合面に凸部や凹部を設けたりしてもよいことも記載されている。【０００４】また、ＣＬＴパネルの接合構造として、例えば特許文献２には、２以上のＣＬＴパネルを互いに交差するように延びかつＣＬＴパネルに埋設される棒状の大径緊結部材および小径緊結部材ならびに接着剤を用いて接合したものが記載されている。


# MeCab による分かち書き

In [7]:
wakati = MeCab.Tagger("-Owakati")

In [8]:
words = wakati.parse(sent)
print(words)

【 背景 技術 】 【 ０ ０ ０ ２ 】 直交 集成 板 （ ＣＬＴ （ Ｃｒｏｓｓ Ｌａｍｉｎａｔｅｄ Ｔｉｍｂｅｒ ） ） は 、 板 の 繊維 方向 が 各層 で 互いに 直交 する よう に 積層 接着 し た パネル で ある 。 ＣＬＴ パネル は 、 直交 積層 で ある ため 、 建築 の 資材 として 使用 さ れ て き た 集成 材 より も 強度 が 安定 し て おり 、 変形 し にくく 、 コンクリート に も 匹敵 する 強度 を 持ち 、 断熱 性 に も 優れ て いる という 特徴 が ある 。 【 ０ ０ ０ ３ 】 従来 、 この ＣＬＴ パネル を 利用 し た 住宅 構造 が 提案 さ れ て いる 。 例えば 、 特許 文献 １ に は 、 ２ 以上 の ＣＬＴ パネル 等 の 箱 型 パネル を 自重 により 支持 さ れる よう に 配置 し 、 特段 の 連結 具 を 使用 する こと なく 建築 物 を 建築 する こと が 記載 さ れ て いる 。 なお 、 特許 文献 １ に は 、 丁 番 等 の 連結 具 を 利用 し たり 、 接合 面 に 凸 部 や 凹部 を 設け たり し て も よい こと も 記載 さ れ て いる 。 【 ０ ０ ０ ４ 】 また 、 ＣＬＴ パネル の 接合 構造 として 、 例えば 特許 文献 ２ に は 、 ２ 以上 の ＣＬＴ パネル を 互いに 交差 する よう に 延び かつ ＣＬＴ パネル に 埋設 さ れる 棒状 の 大 径緊 結 部材 および 小径 緊結 部材 ならびに 接着 剤 を 用い て 接合 し た もの が 記載 さ れ て いる 。 



# GiNZA による固有表現抽出

In [9]:
#抽出したエンティティの種類に対して色を指定する
colors = {"COUNTRY":"#00cc00", "CITY":"#00cc00", "GPE_OTHER":"#00cc00","OCCASION_OTHER":"#00cc00",
          "LOCATION":"#00cc00", "LOCATION_OTHER":"#00cc00","DOMESTIC_REGION":"#00cc00","PROVINCE":"#00cc00",
          "STATION":"#00cc00", "CONTINENTAL_REGION":"#00cc00","THEATER":"#00cc00",

          "TIME":"#adff2f","DATE":"#adff2f","DAY_OF_WEEK":"#adff2f",
          "PERIOD_YEAR":"#adff2f", "PERIOD_MONTH":"#adff2f", "PERIOD_DAY":"#adff2f",

          "FLORA":"#adff99","FLORA_PART":"#adff99",
          "DISH":"#ffeb99","FOOD_OTHER":"#ffeb99",

          "AGE":"#3385ff","N_PERSON":"#3385ff","N_EVENT":"#3385ff","N_LOCATION_OTHER":"#3385ff","RANK":"#3385ff",
          "N_PRODUCT":"#3385ff","":"#3385ff","":"#3385ff","":"#3385ff","MEASUREMENT_OTHER":"#3385ff","PERCENT":"#3385ff",
          "N_ORGANIZATION":"#3385ff", "ORDINAL_NUMBER":"#3385ff", "N_FACILITY":"#3385ff","SPEED":"#3385ff",
          "PHONE_NUMBER":"#3385ff",

          "MONEY":"#ffff00",

          "COMPANY":"#99c2ff", "SCHOOL":"#99c2ff", "INTERNATIONAL_ORGANIZATION":"#99c2ff",
          "GOE_OTHER":"#99c2ff", "SHOW_ORGANIZATION":"#99c2ff","CORPORATION_OTHER":"#99c2ff",

          "CLOTHING":"#ff66a3",
          "PRODUCT_OTHER":"#ff66a3",

          "PERSON":"#c266ff",
          "POSITION_VOCATION":"#ebccff",

          "MUSIC":"#ff7f50", "MOVIE":"#ff7f50", "GAME":"#ff7f50", "SPORT":"#ff7f50", "BOOK":"#ff7f50", 
          "BROADCAST_PROGRAM":"#ff7f50", 

          "ANIMAL_DISEASE":"#cd5c5c"
          }

In [10]:
doc = nlp(sent)
options = {"colors": colors}
displacy.render(doc, style="ent", options=options, jupyter=True)

## ルールの追加

In [11]:

#----- ここに新しいルールを追加する -----#
ruler.add_patterns([
    {"label": "DOCTRINE_METHOD_OTHER", "pattern": "直交集成板"},
    {"label": "DOCTRINE_METHOD_OTHER", "pattern": "通信モジュール"},
    {"label": "DOCTRINE_METHOD_OTHER", "pattern": "大径緊結部材"}])
#-----------------------------------#

nlp.add_pipe(ruler)

In [12]:
doc = nlp(sent)
options = {"colors": colors}
displacy.render(doc, style="ent", options=options, jupyter=True)

In [13]:
for ent in doc.ents:
#     print(ent.label_)
    if ent.label_ in set(["DOCTRINE_METHOD_OTHER","Product_Other"]):
        print(ent)

直交集成板
ＣＬＴ
大径緊結部材


# N-grams

In [21]:
# Bi-gram
bigrams = ngrams(wakati.parse(sent).split(), 2)
for grams in bigrams:
    print(grams)

('【', '背景')
('背景', '技術')
('技術', '】')
('】', '【')
('【', '０')
('０', '０')
('０', '０')
('０', '２')
('２', '】')
('】', '直交')
('直交', '集成')
('集成', '板')
('板', '（')
('（', 'ＣＬＴ')
('ＣＬＴ', '（')
('（', 'Ｃｒｏｓｓ')
('Ｃｒｏｓｓ', 'Ｌａｍｉｎａｔｅｄ')
('Ｌａｍｉｎａｔｅｄ', 'Ｔｉｍｂｅｒ')
('Ｔｉｍｂｅｒ', '）')
('）', '）')
('）', 'は')
('は', '、')
('、', '板')
('板', 'の')
('の', '繊維')
('繊維', '方向')
('方向', 'が')
('が', '各層')
('各層', 'で')
('で', '互いに')
('互いに', '直交')
('直交', 'する')
('する', 'よう')
('よう', 'に')
('に', '積層')
('積層', '接着')
('接着', 'し')
('し', 'た')
('た', 'パネル')
('パネル', 'で')
('で', 'ある')
('ある', '。')
('。', 'ＣＬＴ')
('ＣＬＴ', 'パネル')
('パネル', 'は')
('は', '、')
('、', '直交')
('直交', '積層')
('積層', 'で')
('で', 'ある')
('ある', 'ため')
('ため', '、')
('、', '建築')
('建築', 'の')
('の', '資材')
('資材', 'として')
('として', '使用')
('使用', 'さ')
('さ', 'れ')
('れ', 'て')
('て', 'き')
('き', 'た')
('た', '集成')
('集成', '材')
('材', 'より')
('より', 'も')
('も', '強度')
('強度', 'が')
('が', '安定')
('安定', 'し')
('し', 'て')
('て', 'おり')
('おり', '、')
('、', '変形')
('変形', 'し')
('し', 'にくく')
('にくく', '、')
('、', 'コンクリート')
('コンクリート', 'に')
('に', '

In [20]:
# Tri-gram
trigrams = ngrams(wakati.parse(sent).split(), 3)
for grams in trigrams:
    print(grams)

('【', '背景', '技術')
('背景', '技術', '】')
('技術', '】', '【')
('】', '【', '０')
('【', '０', '０')
('０', '０', '０')
('０', '０', '２')
('０', '２', '】')
('２', '】', '直交')
('】', '直交', '集成')
('直交', '集成', '板')
('集成', '板', '（')
('板', '（', 'ＣＬＴ')
('（', 'ＣＬＴ', '（')
('ＣＬＴ', '（', 'Ｃｒｏｓｓ')
('（', 'Ｃｒｏｓｓ', 'Ｌａｍｉｎａｔｅｄ')
('Ｃｒｏｓｓ', 'Ｌａｍｉｎａｔｅｄ', 'Ｔｉｍｂｅｒ')
('Ｌａｍｉｎａｔｅｄ', 'Ｔｉｍｂｅｒ', '）')
('Ｔｉｍｂｅｒ', '）', '）')
('）', '）', 'は')
('）', 'は', '、')
('は', '、', '板')
('、', '板', 'の')
('板', 'の', '繊維')
('の', '繊維', '方向')
('繊維', '方向', 'が')
('方向', 'が', '各層')
('が', '各層', 'で')
('各層', 'で', '互いに')
('で', '互いに', '直交')
('互いに', '直交', 'する')
('直交', 'する', 'よう')
('する', 'よう', 'に')
('よう', 'に', '積層')
('に', '積層', '接着')
('積層', '接着', 'し')
('接着', 'し', 'た')
('し', 'た', 'パネル')
('た', 'パネル', 'で')
('パネル', 'で', 'ある')
('で', 'ある', '。')
('ある', '。', 'ＣＬＴ')
('。', 'ＣＬＴ', 'パネル')
('ＣＬＴ', 'パネル', 'は')
('パネル', 'は', '、')
('は', '、', '直交')
('、', '直交', '積層')
('直交', '積層', 'で')
('積層', 'で', 'ある')
('で', 'ある', 'ため')
('ある', 'ため', '、')
('ため', '、', '建築')
('、', '建築', 'の')
('建築', 'の', '資材')
(

In [26]:
from ja_stopword_remover.remover import StopwordRemover
import pprint

stopwordRemover = StopwordRemover()

text_list_result = stopwordRemover.remove(wakati.parse(sent).split())
pprint.pprint(text_list_result)

AttributeError: 'str' object has no attribute 'remove'